In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split

In [ ]:
class Node:
    
    def __init__(self, X, y, gini, feature = None, threshold=None, left=None, right=None, value=None):
        self.X = X
        self.y = y
        self.gini = gini
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def check_leaf(self):
        return self.value is not None

In [ ]:
class MyDecisionTreeClassifier:
    
    def __init__(self, min_samples_split = 2, max_depth = 2, feature_number = 0):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.path = None
        self.feature_number = feature_number
    
    def gini(self, groups, classes):
        '''
        A Gini score gives an idea of how good a split is by how mixed the
        classes are in the two groups created by the split.
        
        A perfect separation results in a Gini score of 0,
        whereas the worst case split that results in 50/50
        classes in each group result in a Gini score of 0.5
        (for a 2 class problem).
        '''
        
        pass
    
    def split_data(self, X, y) -> tuple[int, int]:
        
        # test all the possible splits in O(N*F) where N in number of samples
        # and F is number of features

        # return index and threshold value
        
        pass
    
    def build_tree(self, X, y, depth = 0):
        
        # create a root node
        
        # recursively split until max depth is not exeeced
        
        pass
    
    def fit(self, X, y):
        if not self.feature_number:
            self.feature_number = X.shape[1]
        else:
            self.feature_number = min(X.shape[1], self.feature_number)
        self.path = self.build_tree(X, y)
    
    def predict(self, X_test):
        
        # traverse the tree while there is a child
        # and return the predicted class for it, 
        # note that X_test can be a single sample or a batch
        
        return np.array([self.traverse_tree(x, self.path) for x in X_test])
    
    def traverse_tree(self, x, node):
        if node.check_leaf():
            return node.value
        if x[node.feature] <= node.threshold:
            return self.traverse_tree(x, node.left)
        return self.traverse_tree(x, node.right)

    def evaluate(self, X_test, y_test):
        # return accuracy
        classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
        y_pred = classifier.predict(X_test)
        return self.accuracy_scr(y_test, y_pred)
    
    def accuracy_scr(self, y_test, y_pred):
        accuracy = np.sum(y_test == y_pred) / len(y_test)
        return accuracy